# FUNCTION Demo

### Referenz:

- [User-defined functions](https:\learn.microsoft.com\en-us\sql\relational-databases\user-defined-functions\user-defined-functions?view=sql-server-ver16)
- [Create user-defined functions](https:\learn.microsoft.com\en-us\sql\relational-databases\user-defined-functions\create-user-defined-functions-database-engine?view=sql-server-ver16)
    - [Scalar function (scalar UDF)](https:\learn.microsoft.com\en-us\sql\relational-databases\user-defined-functions\create-user-defined-functions-database-engine?view=sql-server-ver16#scalar-function-scalar-udf)
    - [Inline table-valued function (TVF)](https:\learn.microsoft.com\en-us\sql\relational-databases\user-defined-functions\create-user-defined-functions-database-engine?view=sql-server-ver16#inline-table-valued-function-tvf)
    - [Multi-statement table-valued function (MSTVF)](https:\learn.microsoft.com\en-us\sql\relational-databases\user-defined-functions\create-user-defined-functions-database-engine?view=sql-server-ver16#multi-statement-table-valued-function-mstvf)

Funktionen können einzelne Werte (**Skalarwerte**) zurückgeben oder mehrere Werte (**Tabellen**). Beide Arten können Eingabeparameter haben.

Eine Funktion, die nur eine Anweisung enthält, nennt man **Inline-Funktion.** Komplexere Funktionen, die Kontrollstrukturen und mehrere Anweisungen enthalten, heißen **Multi-Statement-Funktionen.**

**Skalarwertfunktionen** werden in SELECT-Klauseln verwendet; **Tabellenwertfunktionen** in FROM-Klauseln.

Hier wird zunächst eine benutzerdefinierte Skalarwertfunktion demonstriert, das heißt: diese Funktion gibt einen einzelnen Wert zurück.

Die folgenden Beispiel-Funktionen verwenden das VIEW <span style="color: rgb(33, 33, 33); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 18px; white-space: pre;">vw_MitarbeiterAbteilungProjektTätigkeit aus der VIEW Demo.</span>

In [ ]:
USE NormalisierungsDemo;

In [ ]:
-- Das soll die Funktion tun, aber für beliebige Projekte
SELECT SUM(CAST(Stunden as int)) AS Gesamtstunden
FROM vw_MitarbeiterAbteilungProjektTätigkeit
WHERE Projekt = 'Kundenumfrage';

In [ ]:
-- So wird die Funktion erstellt
CREATE OR ALTER FUNCTION dbo.fnc_ProjektStunden (@Projekt varchar(60))
RETURNS int
AS
BEGIN
    DECLARE @Gesamtstunden int;
    SELECT @Gesamtstunden = SUM(CAST(Stunden as int))
    FROM vw_MitarbeiterAbteilungProjektTätigkeit
    WHERE Projekt = @Projekt;
    RETURN @Gesamtstunden;
END;

In [ ]:
-- So wird die Funktion verwendet
SELECT dbo.fnc_ProjektStunden('Verkaufsmesse') AS [Gesamtstunden im Projekt Verkaufsmesse]

In [ ]:
-- Funktion zur Berechnung der Personalkosten 
-- für ein als Parameter übergebenes Projekt!
CREATE OR ALTER FUNCTION dbo.fnc_Personalkosten (@Projekt VARCHAR(60))
RETURNS int
AS
BEGIN
    DECLARE @Personalkosten DECIMAL(9,2);
    SELECT @Personalkosten = SUM(CAST(Stunden as DECIMAL(9,2)) * CAST(Stundenlohn AS DECIMAL(9,2)))
    FROM vw_MitarbeiterAbteilungProjektTätigkeit
    WHERE Projekt = @Projekt;
    RETURN @Personalkosten;
END;

In [ ]:
-- Personalkosten des Projekts Verkaufsmesse abrufen
SELECT dbo.fnc_Personalkosten('Verkaufsmesse') AS [Personalkosten im Projekt Verkaufsmesse]

## Tabellenwertfunktion

Ein Funktion kann nicht nur einzelne Werte (=Skalare), sondern auch Tabellen zurückgeben

  

Nachfolgend wird eine Funktion demonstriert, die eine Abteilung als Parameter bekommt und die involvierten Mitarbeiter anzeigt.

In [ ]:
-- Abfrage zur Vorbereitung
SELECT pr.ProjNr, pr.Beschreibung AS Projekt,
	CONCAT(pe.Vorname, ' ', pe.Name) AS Mitarbeiter,
	t.Tätigkeit, aa.Stunden
FROM Projekt AS pr
JOIN Arbeitet_An AS aa ON pr.ProjNr = aa.ProjNr
JOIN Personal AS pe ON aa.PersonalNr = pe.PersonalNr
JOIN Tätigkeit AS t ON aa.TätigkeitsNr = t.TätigkeitsNr
WHERE pr.Beschreibung = 'Kundenumfrage'

In [ ]:
CREATE OR ALTER FUNCTION dbo.fn_PersonalImProjekt (@p AS VARCHAR(120)) RETURNS TABLE
AS 
RETURN
SELECT pr.ProjNr, pr.Beschreibung AS Projekt,
	CONCAT(pe.Vorname, ' ', pe.Name) AS Mitarbeiter,
	t.Tätigkeit, aa.Stunden
FROM Projekt AS pr
JOIN Arbeitet_An AS aa ON pr.ProjNr = aa.ProjNr
JOIN Personal AS pe ON aa.PersonalNr = pe.PersonalNr
JOIN Tätigkeit AS t ON aa.TätigkeitsNr = t.TätigkeitsNr
WHERE pr.Beschreibung = @p;

In [ ]:
-- Zeige Mitarbeiter im Projekt Kundenumfrage
SELECT * FROM dbo.fn_PersonalImProjekt('Kundenumfrage');